# Regresión Softmax

La regresión Softmax (o regresión logística multinomial) es una generalización de la regresión logística para el caso en el que queremos manejar múltiples clases. En la regresión logística, asumíamos que las etiquetas eran binarias: $y^{(i)} \in \{0, 1\}$. Usábamos dicho clasificador para distinguir entre dos tipos de dígitos escritos a mano. La regresión Softmax nos permite manejar $y^{(i)} \in \{1, \dots, K\}$, donde $K$ es el número de clases.

Es más difícil entrenar el modelo con carios valores de puntuación, ya que es difícil diferenciarlos al implementar el algoritmo de Descenso de Gradiente para minimizar la función de coste. Por lo tanto, necesitamos una función que normalice las puntuaciones logit y las haga fácilmente diferenciables. 

Asi que primeramente el modelo de Regresión Softmax calcula una puntuación $s_k(x)$ para cada clase $k$. La ecuación para calcular $s_k(x)$ es ;la siguiente:

$$
s_k(\mathbf{x}) = \mathbf{x}^T \theta^{(k)}
$$

Luego estima la probabilidad de cada clase aplicando la función softmax (también llamada exponencial normalizada) a dichas puntuaciones, esta calcula la exponencial de cada puntuación, y luego las normaliza como se ve en la siguiente ecuacion:

$$
\hat{p}_k = \sigma(\mathbf{s}(\mathbf{x}))_k = \frac{\exp(s_k(\mathbf{x}))}{\sum_{j=1}^{K} \exp(s_j(\mathbf{x}))}
$$

- $K$ es el número de clases.
- $\mathbf{s}(\mathbf{x})$ es un vector que contiene las puntuaciones (scores) de cada clase para la instancia $x$.
- $\sigma(\mathbf{s}(\mathbf{x}))_k$ es la probabilidad estimada de que la instancia $x$ pertenezca a la clase $k$, dado el puntaje de cada clase.

Al igual que el clasificador de Regresión Logística, el clasificador de Regresión Softmax predice la clase con la probabilidad estimada más alta utilizando el operador argmax, que devuelve el valor de la clase que hace que el valor evaluado sea mas alta, se representa como la siguiente ecuacion:

$$
\hat{y} = \underset{k}{\text{argmax}} \ \sigma(\mathbf{s}(\mathbf{x}))_k = \underset{k}{\text{argmax}} \ s_k(\mathbf{x}) = \underset{k}{\text{argmax}} \ \left( \theta^{(k)T} \mathbf{x} \right)
$$

### Función de costo

La función de costo de la Regresión Softmax evalúa qué tan bien (o mal) está funcionando el modelo para clasificar múltiples categorías. Se conoce comúnmente como Entropía Cruzada (Cross-Entropy Loss) o Log Loss Multinomial. Para un conjunto de entrenamiento con $m$ ejemplos y $K$ clases, la función de costo $J(\Theta)$ se define como:

$$
J(\Theta) = - \frac{1}{m} \sum_{i=1}^{m} \sum_{k=1}^{K} y_k^{(i)} \log(\hat{p}_k^{(i)})
$$

- $y_k^{(i)}$ es la probabilidad objetivo de que la $i$-ésima instancia pertenezca a la clase $k$. En general, es igual a 1 o 0, dependiendo de si la instancia pertenece a la clase o no

### Descenso de gradiente

Para realizar el algoritmo del descenso de la gradiente en esta regresion, necesitamos calcular la gradiente de la funcion de costo, planteada anteriormente. Asi que el vector de gradiente de esta función de costo con respecto a $\theta^{(k)}$ viene dado por la siguiente ecuacion matricial:

$$
\nabla_{\theta^{(k)}} J(\mathbf{\Theta}) = \frac{1}{m} \sum_{i=1}^{m} \left( \hat{p}_k^{(i)} - y_k^{(i)} \right) \mathbf{x}^{(i)}
$$

Ya teniendo la gradiente, el descenso de la gradiente estaria dada por la siguiente ecuacion:

$$
\theta^{(k)} := \theta^{(k)} - \alpha \nabla_{\theta^{(k)}} J(\theta)
$$



In [22]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression

In [20]:
def add_bias(X):
    X_b = np.c_[np.ones((X.shape[0], 1)), X]
    return X_b

def softmax(Z):
    expZ = np.exp(Z - np.max(Z, axis=1, keepdims=True)) # Evita el overflow
    return expZ / np.sum(expZ, axis=1, keepdims=True)

def gradientDescent(lr,n_epochs,X, y, K):
    m = X.shape[0]
    n = X.shape[1]
    theta = np.random.randn(n,K)

    for i in range(n_epochs):
        P = softmax(X @ theta)
        gradients = 1/m * X.T @ (P - y)
        theta -= lr * gradients
        
    return theta

In [21]:
m = 100   # Cantidad de datos
n = 3     # Features
K = 3     # Número de clases

X = 2 * np.random.randn(m, n)
X_b = add_bias(X)

true_theta = np.array([
    [1.0, -2.0, 2.5],
    [2.0,  1.0, -1.5],
    [-1.0, 0.5, 1.0],
    [0.5, -0.5, 1.0]
])

scores = X_b @ true_theta
y_labels = np.argmax(softmax(scores), axis=1)

Y = np.eye(K)[y_labels]

theta = gradientDescent(0.1, 2000, X_b, Y, K)

print("\nParámetros aprendidos (θ):")
print(theta)



Parámetros aprendidos (θ):
[[ 1.45200379 -0.78303333  4.54764552]
 [ 2.96389217 -0.10875816 -3.01754123]
 [-1.51671955  1.24066866  1.96553526]
 [ 0.11225326 -1.05249668  1.29838315]]


### Implementacion con sklearn

In [24]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

m, n, K = 100, 3, 3
X = 2 * np.random.randn(m, n)
 
true_theta = np.array([
    [1.0, -2.0, 2.5], [2.0, 1.0, -1.5],
    [-1.0, 0.5, 1.0], [0.5, -0.5, 1.0]
])

X_b = add_bias(X)
scores = X_b @ true_theta
y = np.argmax(scores, axis=1)

softmax_reg = LogisticRegression(solver='lbfgs', C=1e5)

softmax_reg.fit(X, y)

y_pred = softmax_reg.predict(X)

# --- Resultados ---
print("--- Resultados Sklearn ---")
print(f"Precisión: {accuracy_score(y, y_pred) * 100:.2f}%")
print("\nIntercepto (Bias aprendido):")
print(softmax_reg.intercept_)
print("\nCoeficientes (Pesos aprendidos):")
print(softmax_reg.coef_.T)

--- Resultados Sklearn ---
Precisión: 100.00%

Intercepto (Bias aprendido):
[  6.14135859 -51.27568435  45.13432576]

Coeficientes (Pesos aprendidos):
[[ 40.59541507  16.6838924  -57.27930747]
 [-28.6748934    6.2853684   22.389525  ]
 [  2.65615043 -18.22750125  15.57135082]]
